In [9]:
import os
import sys
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
import cv2
import argparse

# Root directory of the project

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

# sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))
# import coco

In [10]:
def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

In [34]:
# based on https://github.com/matterport/Mask_RCNN/blob/master/mrcnn/visualize.py
# and https://github.com/markjay4k/Mask-RCNN-series/blob/887404d990695a7bf7f180e3ffaee939fbd9a1cf/visualize_cv.py

# colors = [tuple(255 * np.random.rand(3)) for _ in range(3)]
colors = [(245, 178, 137),(107, 139, 202),(186, 233, 154)]

def display_instances(image, boxes, masks, class_ids, class_names, scores=None):
    assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]
    
    N = boxes.shape[0]

    for i in range(N):
        if not np.any(boxes[i]):
            continue
        
        y1, x1, y2, x2 = boxes[i]
        label = class_names[class_ids[i]]
        
        color = colors[class_ids[i]-1]
        
        score = scores[i] if scores is not None else None
        caption = "{} {:.3f}".format(label, score) if score else label

        # Mask
        mask = masks[:, :, i]
        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2)
    return image

In [28]:
# define the prediction configuration
class PredictionConfig(Config):
    # define the name of the configuration
    NAME = "kangaroo_cfg"
    # number of classes (background + kangaroo)
    NUM_CLASSES = 1 + 3
    # simplify GPU config
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

In [24]:
# parser = argparse.ArgumentParser(description='MaskRCNN Video Object Detection/Instance Segmentation')
# parser.add_argument('-v', '--video_path', type=str, default='', help='Path to video. If None camera will be used')
# parser.add_argument('-sp', '--save_path', type=str, default='', help= 'Path to save the output. If None output won\'t be saved')
# parser.add_argument('-s', '--show', default=True, action="store_false", help='Show output')
# args = parser.parse_args()

args ={
    "video_path":"./dataset/mathikere/VID_20210323_183723.mp4",
    "save_path":"output.avi",
    "MODEL_PATH":"./models/mathikere/mask_rcnn_kangaroo_cfg_0005.h5",
    "show":True 
}

In [15]:
class_names = ['BG',"person","twowheeler","fourwheeler" ]
config = PredictionConfig()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir="./", config=config)

# Load weights trained on MS-COCO
model.load_weights(args["MODEL_PATH"], by_name=True)

In [36]:
count = 0
if args["video_path"] != '':
    cap = cv2.VideoCapture(args["video_path"])
else:
    cap = cv2.VideoCapture(0)

if args["save_path"]:
    width = int(cap.get(3))
    height = int(cap.get(4))
    fps = 3 #cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(args["save_path"], cv2.VideoWriter_fourcc('M','J','P','G'), fps, (width, height))

try:
    while cap.isOpened():
        ret, image = cap.read()
        if ret:
            results = model.detect([image], verbose=0)
            r = results[0]
            image = display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

            if False and args["show"]:
                cv2.imshow('MaskRCNN Object Detection/Instance Segmentation', image)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            if args["save_path"]:
                out.write(image)
                count+=1
                print(count)
                if count==15:
                    break
        else:
            break
except Exception as e:
    print(e)
finally:
    cap.release()
    if args["save_path"]:
        out.release()
    cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
